In [1]:
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from dotenv import load_dotenv
from typing import List
from pprint import pprint
import json

def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()
    return prompt_template

if __name__ == "__main__":

    load_dotenv()
        
    llm = ChatOpenAI(temperature=0.1, 
                     max_tokens=500, 
                     model="gpt-4",
                     verbose = True)
        
    # prompt_template = ChatPromptTemplate.from_template(
    #     template=read_prompt_template('prompts/1. translate.txt')
    # )
    src = {}
    src["src"] = ["Feeling hungry, we grabbed a pizza from the fridge and microwaved it.", 
                  "The video chronologically shows the son's finger stuck in a hole of the dog house, the rescue process by paramedics, and the conclusion of the entire incident."]
    
    prompt_template = ChatPromptTemplate.from_messages( [
        ("system", read_prompt_template('prompts/1. translate.txt')),
        ("human", json.dumps(src).replace('{','{{').replace('}','}}')),
        ] )

    print(prompt_template)

    # CHAINING
    chain = LLMChain(
        llm = llm, 
        prompt = prompt_template, 
        output_key = "output",
        verbose = True
    )
    
    # make prompt with template
    req = {}
    req['src_lang'] = 'english'
    req['dst_lang'] = 'korean'
    
    
    # request 
    result = chain(req)

    # check response
    print("# Raw : ")
    pprint(result)

    
    dst = json.loads(result['output'])['dst']
    
    
    

input_variables=['dst_lang', 'src_lang'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dst_lang', 'src_lang'], template='<task>\ntranslate {src_lang} into {dst_lang} considering the below : \n\n1. make three different (but same meaning, same tone, same style) sentences per a {src_lang} sentence\n2. translated sentence should be accurate, natural,fluent, contextual-consistent, factual-consistent, cutural-appropriate,  \n3. give me your ouput as a json object \n4. give me only the json object\n</task>\n\n<input example>\n{{\n\t"src" : ["sample given {src_lang} sentence#1" , "sample given {src_lang} sentence#2"]\n}}\n</input example>\n\n<output example>\n{{\n\t"dst":[\n\t\t[sample transted {dst_lang} sentence for #1", "sample translated {dst_lang} sentence for #1", "sample translated {dst_lang} sentence for #1"],\n\t\t[sample transted {dst_lang} sentence for #2", "sample translated {dst_lang} sentence for #2", "sample translated {dst_lang} sentence for #2"]

In [23]:
dst = json.loads(result['output'])['dst']

print(len(src['src']) , len(dst) , len(dst[0]), len(dst[1]))

2 2 3 3


In [26]:
json.dumps(src) # serializa object to json formatted string

# json.loads # deserialize json formatted strint to python object


'{"src": ["Feeling hungry, we grabbed a pizza from the fridge and microwaved it.", "The video chronologically shows the son\'s finger stuck in a hole of the dog house, the rescue process by paramedics, and the conclusion of the entire incident."]}'

In [ ]:
json.load